In [132]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from glob import glob

In [133]:
#Start off by reading the dir path and file names for each ticker csv
ticker_dir_list = glob("data/daily_20230216/*.csv")

In [134]:
ticker_dir = ticker_dir_list[0]
firstIndex = ticker_dir.rfind('\\') + 1
lastIndex = ticker_dir.rfind('.')
ticker = ticker_dir[firstIndex:lastIndex]

In [135]:
ticker_df = pd.read_csv(ticker_dir, parse_dates=['timestamp'])
ticker_df = ticker_df.rename(columns={'index':'timestamp', 
                          '1. open':'open',
                          '2. high':'high',
                          '3. low':'low',
                          '4. close':'close',
                          '5. adjusted close':'adjusted close',
                          '6. volume':'volume'}).drop(columns=['7. dividend amount', '8. split coefficient'])
                                                                                 
# Sort by date
ticker_df.sort_values(by=['timestamp'], inplace=True)  
# Reassign indices
ticker_df = ticker_df.reset_index().drop(columns=['index'])

In [136]:
# Find change in price for a day and a month
n = 1
ticker_df['price_delta_daily'] = ticker_df['adjusted close'].diff(n)
ticker_df['price_delta_monthly'] = ticker_df['adjusted close'].diff(n*30)

In [137]:
ticker_df['buy_signal_daily'] = ticker_df['price_delta_daily'].transform(np.sign).shift(-1)
ticker_df['buy_signal_monthly'] = ticker_df['price_delta_monthly'].transform(np.sign).shift(-1)

# Indicators

## Bollinger Bands $\overline{C_{20}} \pm 2 \cdot {SD}_{20}$

In [138]:
n_bollinger = 20
n_of_std = 2

ticker_df['mean_20'] = ticker_df['adjusted close'].rolling(n_bollinger).mean()
ticker_df['stdev_20'] = ticker_df['adjusted close'].rolling(n_bollinger).std()
ticker_df['upper_bollinger'] = ticker_df['mean_20'] + (n_of_std * ticker_df['stdev_20'])
ticker_df['lower_bollinger'] = ticker_df['mean_20'] - (n_of_std * ticker_df['stdev_20'])

## Force Index $(C_t - C_{t-1})\cdot V_t$

In [139]:
ticker_df['force_index'] = ticker_df['price_delta_daily'] * ticker_df['volume']

## Force Index 5 day SMA

In [140]:
ticker_df['force_index_sma_5'] = ticker_df['force_index'].rolling(5).mean()

## Force Index 13 day SMA

In [141]:
ticker_df['force_index_sma_13'] = ticker_df['force_index'].rolling(13).mean()

## Closing Price Relative Change (Daily)

In [142]:
ticker_df['price_rel_change'] = ticker_df['price_delta_daily'] / ticker_df['adjusted close']

## Volume Relative Change (Daily)

In [143]:
ticker_df['vol_rel_change'] = ticker_df['volume'].diff() / ticker_df['volume']

## Negative Volume Index

In [145]:
nvi_rate = ticker_df[ticker_df['vol_rel_change'] < 0]['vol_rel_change'].abs() * ticker_df['price_rel_change']

In [150]:
nvi_rate.fillna(0).max()

inf

## Relative Strength Index (RSI) (Not finished)

$$RSI=100 - \frac{100}{1 + RS}$$

In [14]:
# Averaging Window for RSI     
n_rsi = 14

# Separate up and down days
up_df, down_df = ticker_df[['price_delta_daily']].copy(), ticker_df[['price_delta_daily']].copy()

up_df.loc[(up_df['price_delta_daily'] < 0), 'price_delta_daily'] = 0


In [95]:
ticker_df

,timestamp,open,high,low,close,adjusted close,volume,price_delta_daily,price_delta_monthly,buy_signal_daily,buy_signal_monthly,mean_20,stdev_20,upper_bollinger,lower_bollinger,force_index,force_index_sma_5,force_index_sma_13,price_rel_change,volume_rel_change
0,2008-01-29,9.50,9.990,8.5700,8.7500,1.711889,1489000,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2008-01-30,8.75,9.150,8.3000,8.5000,1.662978,219000,-0.048911,NaN,1.0,NaN,NaN,NaN,NaN,NaN,-10711.535327,NaN,NaN,-0.029412,-5.799087
2,2008-01-31,8.49,10.300,8.4900,9.5500,1.868405,182300,0.205427,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,37449.288302,NaN,NaN,0.109948,-0.201317
3,2008-02-01,9.93,9.940,9.5000,9.5100,1.860579,28200,-0.007826,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,-220.686974,NaN,NaN,-0.004206,-5.464539
4,2008-02-04,9.50,9.712,9.5000,9.5000,1.858623,8300,-0.001956,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,-16.238492,NaN,NaN,-0.001053,-2.397590
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3782,2023-02-10,1.70,1.800,1.6501,1.7400,1.740000,11384,0.035000,0.4331,1.0,1.0,1.489485,0.117821,1.725126,1.253844,398.440000,261.8404,140.967092,0.020115,-0.294624
3783,2023-02-13,1.79,1.820,1.7482,1.7700,1.770000,12984,0.030000,0.4200,1.0,1.0,1.509485,0.129811,1.769106,1.249864,389.520000,-110.0756,171.036769,0.016949,0.123229
3784,2023-02-14,1.85,1.850,1.7700,1.7805,1.780500,8289,0.010500,0.5072,1.0,1.0,1.529510,0.139327,1.808164,1.250856,87.034500,406.2869,132.741746,0.005897,-0.566413
3785,2023-02-15,1.79,1.860,1.7900,1.8500,1.850000,10826,0.069500,0.5900,1.0,1.0,1.550510,0.154378,1.859266,1.241754,752.407000,487.5983,218.309592,0.037568,0.234343
